# Task 2: Recommendation Engine

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the Used Cars dataset; there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* Please consider this notebook as an example and not to set specific requirements. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [1]:
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

## Load the Data

For this example, we use a simplified version of the dataset with only 100 data sample, each with only 6 features

In [69]:
df = pd.read_csv('./data/train.csv')

title_ls = df.title.tolist()
description_ls = df.description.tolist()
features_ls = df.features.tolist()
accessories_ls = df.accessories.tolist()

description_ls = [str(title_ls[i])+" "+str(description_ls[i])+" "+str(features_ls[i])+" "+str(accessories_ls[i])
                  for i in range(len(title_ls))]

df = pd.concat((df, pd.DataFrame(description_ls, columns=['desc_concat'])), axis=1)

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(description_ls)

tfidf_score_arr = np.array(vectors.todense())

## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [77]:
def get_top_recommendations(row, train_df, vectorizer, tfidf_score_arr, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = 1
    
    # Extract all **kwargs input parameters
    # and set the used paramaters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value
            
       
    #####################################################
    ## Compute your recommendations
    question = row['title'] + " " + row['description']+ " " + row['features']+ " " + row['accessories']
    
    tfidf_question = vectorizer.transform([question])

    query_mask = np.where(np.array(tfidf_question.todense())>0, 1, 0)

    doc_score = np.sum(query_mask * tfidf_score_arr, axis=1)

    doc_rank_idx = doc_score.argsort()[::-1]

    doc_rank_idx = doc_rank_idx[:k]

    doc_score = doc_score[doc_rank_idx[:k]]
    
    df_result = train_df.loc[doc_rank_idx]
    
    df_result['score'] = doc_score
    # Return the dataset with the k recommendations
    return df_result


## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [80]:
# Pick a row id of choice
# row_id = 10
row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
pd.DataFrame([row])

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price,desc_concat
20,1016319,Volkswagen Scirocco 2.0A TSI Sunroof (COE till...,volkswagen,scirocco,nothing like a 2.0tsi ea888 engine in a sciroc...,2009.0,NaN,05-oct-2009,sports car,"coe car, premium ad car, low mileage car",...,30382.0,25271.0,NaN,NaN,uncategorized,"powerful yet economical 2.0tsi ea888 engine, p...","volk ze40 wheels, revo intake, revo stage 2 fl...",NaN,79000.0,Volkswagen Scirocco 2.0A TSI Sunroof (COE till...


## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [81]:
k = 3

df_recommendations = get_top_recommendations(row, df, vectorizer, tfidf_score_arr, k=k)

df_recommendations.head(k)

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price,desc_concat,score
20,1016319,Volkswagen Scirocco 2.0A TSI Sunroof (COE till...,volkswagen,scirocco,nothing like a 2.0tsi ea888 engine in a sciroc...,2009.0,NaN,05-oct-2009,sports car,"coe car, premium ad car, low mileage car",...,25271.0,NaN,NaN,uncategorized,"powerful yet economical 2.0tsi ea888 engine, p...","volk ze40 wheels, revo intake, revo stage 2 fl...",NaN,79000.0,Volkswagen Scirocco 2.0A TSI Sunroof (COE till...,7.198566
12961,1029908,Volkswagen Scirocco GT 2.0A TSI (COE till 04/2...,volkswagen,scirocco,properly done up scirocco 2.0tsi with chilli r...,2009.0,NaN,26-sep-2009,sports car,"coe car, premium ad car",...,27436.0,NaN,NaN,uncategorized,"2.0tsi engine, reliable 6-speed dq250 wet clut...","legal fox exhaust, mst intake, full r kit, bil...",NaN,82300.0,Volkswagen Scirocco GT 2.0A TSI (COE till 04/2...,3.625302
994,1024986,Volkswagen Scirocco 2.0A TSI (COE till 02/2030),volkswagen,scirocco,100% loan available. all wear and tear done up...,2009.0,NaN,11-feb-2010,sports car,"coe car, premium ad car",...,30514.0,NaN,NaN,uncategorized,"powerful yet economical 2.0tsi ea888 engine, p...","stage 2 tune, d1 spec rs 4 pot bbk, abt full b...",NaN,89100.0,Volkswagen Scirocco 2.0A TSI (COE till 02/2030...,3.124378
